In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import requests
from bs4 import BeautifulSoup
import time 
from tqdm import tqdm
import random
import pandas as pd

In [2]:
def get_sitemap_urls(sitemap_url):
    response = requests.get(sitemap_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        urls = [loc.text for loc in soup.find_all('loc')]
        return urls
    else:
        print(f"Failed to fetch sitemap. Status Code: {response.status_code}")
        return []

In [3]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [4]:
def scrollWindow():
    initial_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        driver.implicitly_wait(2) 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == initial_height:
            break
        initial_height = new_height
    time.sleep(2)   

In [5]:
def get_product_info(URL):
    driver.get(URL+'#reviews')
    time.sleep(2)
    scrollWindow()
    wait = WebDriverWait(driver, 10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    p_element = soup.find('h1')
    Product = p_element.text
    Subp_element = soup.find('h2', class_="text-sm italic text-secondary")
    SubP = Subp_element.text if Subp_element else None
    rating_element = soup.find('span',class_="stamped-summary-text-1")
    rating = rating_element.text if rating_element else '0'
    review_counting_element = soup.find('span', class_="mr-5 text-sm")
    review_count= review_counting_element.text if review_counting_element else '0 reviews'
    category_element = soup.find_all('div',class_="flex flex-wrap gap-3")
    df= []
    if review_count != '0 reviews' and rating == '0':
        driver.get(URL+'#reviews')
        time.sleep(30)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        rating_element = soup.find('span',class_="stamped-summary-text-1")
        rating = rating_element.text if rating_element else '0'
    else:
        pass
    if len(category_element)!=0:
        Sbutton = category_element[0].find_all('button')
        for i in range(len(Sbutton)):
            Category = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div[2]/div/form/div[3]/section[1]/div/button[{}]'.format(i+1) )))  
            try:
                Category.click()
            except Exception as e:
                driver.execute_script("arguments[0].click();", Category)
            C = Category.text
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            category_element = soup.find_all('div',class_="flex flex-wrap gap-3")
            Sbutton = category_element[0].find_all('button')
            if len(category_element)==1:
                Pbutton=[]
            else:
                Pbutton = category_element[1].find_all('button')
            if Pbutton:
                for j in range(len(Pbutton)):
                    Pack = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div[2]/div/form/div[3]/section[2]/div/button[{}]'.format(j+1) ))) 
                    try:
                        Pack.click()
                    except Exception as e:
                        driver.execute_script("arguments[0].click();", Pack)
                    Pa = Pack.text
                    try:
                        RegularPrice = driver.find_element(By.XPATH, '/html/body/div[1]/main/div/div/div[2]/div/form/div[2]/div/s')
                        RegularPr = RegularPrice.text
                        
                    
                    except Exception as e:
                        RegularPr = None
                    
                    SalePrice = driver.find_element(By.XPATH, '/html/body/div[1]/main/div/div/div[2]/div/form/div[2]/div/div/span' )
                                                               
                    SalePr = SalePrice.text
                    SalePr = SalePr[:-2] + "." + SalePr[-2:]
                    row={
                        'ProductURL': URL,
                        'ProductName': Product,
                        'SubHeading': SubP,
                        'Availability': 'InStock',
                        'Category': C.split('\n')[0], 
                         'Pack'   : Pa.split('\n')[0],
                        'RegularPrice'  : RegularPr,
                         'SalePrice'  : SalePr,
                        'ReviewCount': review_count,
                        'Rating': rating

                        }
                    df.append(row)
            else:
                try:
                    RegularPrice = driver.find_element(By.XPATH, '/html/body/div[1]/main/div/div/div[2]/div/form/div[2]/div/s')
                    RegularPr = RegularPrice.text
                    

                except Exception as e:
                    RegularPr = None
                SalePrice = driver.find_element(By.XPATH, '/html/body/div[1]/main/div/div/div[2]/div/form/div[2]/div/div/span')
                                                          
                SalePr = SalePrice.text
                SalePr = SalePr[:-2] + "." + SalePr[-2:]
                row={
                    'ProductURL': URL,
                    'ProductName': Product,
                    'SubHeading': SubP,
                    'Availability': 'InStock',
                    'Category': C.split('\n')[0], 
                     'Pack'   : None,
                    'RegularPrice'  : RegularPr,
                     'SalePrice'  : SalePr,
                    'ReviewCount': review_count,
                    'Rating': rating

                    }
                df.append(row)
    else:
        row={
                    'ProductURL': URL,
                    'ProductName': Product,
                    'SubHeading': SubP,
                    'Availability': 'SoldOut',
                    'Category': None, 
                     'Pack'   : None,
                    'RegularPrice'  : None,
                     'SalePrice'  : None,
                    'ReviewCount': review_count,
                    'Rating': rating

                    }
        df.append(row)

    return df

In [6]:
allowed_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/14.1.2",
    "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"   
]

In [7]:
sitemap_url = 'https://www.fast-growing-trees.com/sitemap_products_1.xml?from=1532732702772&to=7236252434494'
sitemap_urls = get_sitemap_urls(sitemap_url)
Pro_URL= []
for i in sitemap_urls:
    if 'https://www.fast-growing-trees.com/products' in i:
        Pro_URL.append(i)
df = pd.DataFrame(Pro_URL)
df.to_csv('Links.csv',index=False)

C:\Users\Rahul Manocha\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [8]:
Product_details1=[]
Product_details2=[]
Product_details3=[]
Product_details4=[]
Product_details5=[]
Product_details6=[]
Product_details7=[]
Product_details8=[]
Product_details9=[]
Product_details10=[]
Product_details11=[]
Product_details12=[]
Product_details13=[]
Product_details14=[]
Product_details15=[]
Product_details16=[]

In [27]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1000:1100], desc="GettingInfo", unit="URL"):
    print(i)
    df = get_product_info(i)
    Product_details11.extend(df)
    print(df)
daf = pd.DataFrame(Product_details11)
daf.to_csv('FastGrowingTrees11.csv',index=False)
driver.quit()

GettingInfo:   0%|                                                                            | 0/100 [00:00<?, ?URL/s]

https://www.fast-growing-trees.com/products/bloom-bash-hollywood-hibiscus


GettingInfo:   1%|▋                                                                   | 1/100 [00:10<17:58, 10.90s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/bloom-bash-hollywood-hibiscus', 'ProductName': 'Bloom Bash® Hollywood® Hibiscus', 'SubHeading': "<p><i>Hibiscus rosa-sinensis </i>'Bloom Bash'</p>", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/midnight-marvel-hibiscus


GettingInfo:   2%|█▎                                                                  | 2/100 [00:20<16:25, 10.06s/URL]

...

[{'ProductURL': 'https://www.fast-growing-trees.com/products/all-in-one-citrus-starter-kit', 'ProductName': 'All-in-One Citrus Starter Kit', 'SubHeading': 'Citrus x meyeri; Citrus aurantifolia; Citrus mitis', 'Availability': 'InStock', 'Category': '1 Gallon Citrus Starter Kit', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$219.95', 'ReviewCount': '12 reviews', 'Rating': '4.6'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/all-in-one-citrus-starter-kit', 'ProductName': 'All-in-One Citrus Starter Kit', 'SubHeading': 'Citrus x meyeri; Citrus aurantifolia; Citrus mitis', 'Availability': 'InStock', 'Category': '1 Gallon Citrus Starter Kit with Citrus Care Kit', 'Pack': None, 'RegularPrice': '$169.95', 'SalePrice': '$159.95', 'ReviewCount': '12 reviews', 'Rating': '4.6'}]
https://www.fast-growing-trees.com/products/ficus-ruby-variegated-rubber-plant


GettingInfo:  99%|██████████████████████████████████████████████████████████████████▎| 99/100 [13:51<00:09,  9.29s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/ficus-ruby-variegated-rubber-plant', 'ProductName': 'Ficus Ruby (Variegated Rubber Plant)', 'SubHeading': "Ficus elastica 'Ruby'", 'Availability': 'InStock', 'Category': '6 inch', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$34.95', 'ReviewCount': '30 reviews', 'Rating': '3.7'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/ficus-ruby-variegated-rubber-plant', 'ProductName': 'Ficus Ruby (Variegated Rubber Plant)', 'SubHeading': "Ficus elastica 'Ruby'", 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$69.95', 'ReviewCount': '30 reviews', 'Rating': '3.7'}]
https://www.fast-growing-trees.com/products/prairie-fire-sedge


GettingInfo: 100%|██████████████████████████████████████████████████████████████████| 100/100 [13:57<00:00,  8.37s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/prairie-fire-sedge', 'ProductName': 'Prairie Fire Sedge', 'SubHeading': 'Carex testacea', 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]


In [28]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1100:1200], desc="GettingInfo", unit="URL"):
    print(i)
    df = get_product_info(i)
    Product_details12.extend(df)
    print(df)
daf = pd.DataFrame(Product_details12)
daf.to_csv('FastGrowingTrees12.csv',index=False)
driver.quit()

GettingInfo:   0%|                                                                            | 0/100 [00:00<?, ?URL/s]

https://www.fast-growing-trees.com/products/free-berry-bush-care-kit


GettingInfo:   1%|▋                                                                   | 1/100 [00:07<13:05,  7.94s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/free-berry-bush-care-kit', 'ProductName': 'Free Berry Bush Care Kit', 'SubHeading': '', 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/monstera-and-fiddle-leaf-fig-combo


GettingInfo:   2%|█▎                                                                  | 2/100 [00:14<11:40,  7.14s/URL]

...

[{'ProductURL': 'https://www.fast-growing-trees.com/products/black-dragon-cryptomeria', 'ProductName': 'Black Dragon Cryptomeria', 'SubHeading': "Cryptomeria japonica 'Black Dragon'", 'Availability': 'InStock', 'Category': '1 gallon', 'Pack': 'Single', 'RegularPrice': None, 'SalePrice': '$69.95', 'ReviewCount': '6 reviews', 'Rating': '4.7'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/black-dragon-cryptomeria', 'ProductName': 'Black Dragon Cryptomeria', 'SubHeading': "Cryptomeria japonica 'Black Dragon'", 'Availability': 'InStock', 'Category': '1 gallon', 'Pack': '4-Pack', 'RegularPrice': '$279.80', 'SalePrice': '$237.83', 'ReviewCount': '6 reviews', 'Rating': '4.7'}]
https://www.fast-growing-trees.com/products/morning-light-miscanthus


GettingInfo:  99%|██████████████████████████████████████████████████████████████████▎| 99/100 [17:15<00:09,  9.18s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/morning-light-miscanthus', 'ProductName': 'Morning Light Miscanthus', 'SubHeading': "Miscanthus sinensis 'Morning Light'", 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': 'Single', 'RegularPrice': None, 'SalePrice': '$84.95', 'ReviewCount': '2 reviews', 'Rating': '4.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/morning-light-miscanthus', 'ProductName': 'Morning Light Miscanthus', 'SubHeading': "Miscanthus sinensis 'Morning Light'", 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': '4-Pack', 'RegularPrice': '$339.80', 'SalePrice': '$288.83', 'ReviewCount': '2 reviews', 'Rating': '4.0'}]
https://www.fast-growing-trees.com/products/chandler-walnut-tree


GettingInfo: 100%|██████████████████████████████████████████████████████████████████| 100/100 [17:27<00:00, 10.47s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/chandler-walnut-tree', 'ProductName': 'Chandler Walnut Tree', 'SubHeading': 'Juglans regia ‘Chandler’', 'Availability': 'InStock', 'Category': '3-4 feet', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$229.95', 'ReviewCount': '21 reviews', 'Rating': '4.9'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/chandler-walnut-tree', 'ProductName': 'Chandler Walnut Tree', 'SubHeading': 'Juglans regia ‘Chandler’', 'Availability': 'InStock', 'Category': '4-5 feet', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$249.95', 'ReviewCount': '21 reviews', 'Rating': '4.9'}]


In [29]:
Product_details8=[]
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1200:1300], desc="GettingInfo", unit="URL"):
    print(i)
    df = get_product_info(i)
    Product_details13.extend(df)
    print(df)
daf = pd.DataFrame(Product_details13)
daf.to_csv('FastGrowingTrees13.csv',index=False)
driver.quit()

GettingInfo:   0%|                                                                            | 0/100 [00:00<?, ?URL/s]

https://www.fast-growing-trees.com/products/carolina-jasmine


GettingInfo:   1%|▋                                                                   | 1/100 [00:13<22:21, 13.55s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/carolina-jasmine', 'ProductName': 'Carolina Jasmine', 'SubHeading': 'Gelsemium sempervirens', 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': 'Single', 'RegularPrice': None, 'SalePrice': '$129.95', 'ReviewCount': '42 reviews', 'Rating': '4.9'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/carolina-jasmine', 'ProductName': 'Carolina Jasmine', 'SubHeading': 'Gelsemium sempervirens', 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': '4-Pack', 'RegularPrice': '$519.80', 'SalePrice': '$436.63', 'ReviewCount': '42 reviews', 'Rating': '4.9'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/carolina-jasmine', 'ProductName': 'Carolina Jasmine', 'SubHeading': 'Gelsemium sempervirens', 'Availability': 'InStock', 'Category': '3 gallon with Trellis', 'Pack': 'Single', 'RegularPrice': None, 'SalePrice': '$109.95', 'ReviewCount': '42 reviews', 'Rating': '4.9'}, {'ProductURL': 'https://www.fast-

GettingInfo:   2%|█▎                                                                  | 2/100 [00:20<16:07,  9.87s/URL]

...

[{'ProductURL': 'https://www.fast-growing-trees.com/products/yuzu-tree', 'ProductName': 'Yuzu Tree', 'SubHeading': 'Citrus junos', 'Availability': 'InStock', 'Category': '1-2 feet', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$129.95', 'ReviewCount': '0 reviews', 'Rating': '5.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/yuzu-tree', 'ProductName': 'Yuzu Tree', 'SubHeading': 'Citrus junos', 'Availability': 'InStock', 'Category': '2-3 feet', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$139.95', 'ReviewCount': '0 reviews', 'Rating': '5.0'}]
https://www.fast-growing-trees.com/products/baby-blue-spruce-tree


GettingInfo:  99%|██████████████████████████████████████████████████████████████████▎| 99/100 [13:22<00:08,  8.87s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/baby-blue-spruce-tree', 'ProductName': 'Baby Blue Spruce Tree', 'SubHeading': "Picea pungens 'Baby Blue'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '27 reviews', 'Rating': '5.0'}]
https://www.fast-growing-trees.com/products/skylark-california-lilac


GettingInfo: 100%|██████████████████████████████████████████████████████████████████| 100/100 [13:29<00:00,  8.10s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/skylark-california-lilac', 'ProductName': 'Skylark California Lilac', 'SubHeading': "Ceanothus thyrsiflorus 'Skylark'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]


In [30]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1300:1400], desc="GettingInfo", unit="URL"):
    print(i)
    df = get_product_info(i)
    Product_details14.extend(df)
    print(df)
daf = pd.DataFrame(Product_details14)
daf.to_csv('FastGrowingTrees14.csv',index=False)
driver.quit()

GettingInfo:   0%|                                                                            | 0/100 [00:00<?, ?URL/s]

https://www.fast-growing-trees.com/products/creeping-mountain-lilac


GettingInfo:   1%|▋                                                                   | 1/100 [00:07<13:07,  7.96s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/creeping-mountain-lilac', 'ProductName': 'Creeping Mountain Lilac', 'SubHeading': 'Ceanothus ‘Joyce Coulter’', 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/housewarming-kit


GettingInfo:   2%|█▎                                                                  | 2/100 [00:15<13:02,  7.99s/URL]

...

[{'ProductURL': 'https://www.fast-growing-trees.com/products/double-delight-nectarine-tree-ca', 'ProductName': 'Double Delight Nectarine Tree', 'SubHeading': "Prunus persica var. neupersica 'Double Delight'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/ohenry-peach-tree-ca


GettingInfo:  99%|██████████████████████████████████████████████████████████████████▎| 99/100 [15:49<00:07,  7.30s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/ohenry-peach-tree-ca', 'ProductName': "O'Henry Peach Tree", 'SubHeading': "Prunus persica 'O'Henry'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '5 reviews', 'Rating': '4.8'}]
https://www.fast-growing-trees.com/products/ruby-red-grapefruit-tree-fl


GettingInfo: 100%|██████████████████████████████████████████████████████████████████| 100/100 [15:57<00:00,  9.57s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/ruby-red-grapefruit-tree-fl', 'ProductName': 'Ruby Red Grapefruit Tree', 'SubHeading': "Citrus x paradise 'Ruby Red'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '3 reviews', 'Rating': '5.0'}]


In [31]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1400:1500], desc="GettingInfo", unit="URL"):
    print(i)
    df = get_product_info(i)
    Product_details15.extend(df)
    print(df)
daf = pd.DataFrame(Product_details15)
daf.to_csv('FastGrowingTrees15.csv',index=False)
driver.quit()

GettingInfo:   0%|                                                                            | 0/100 [00:00<?, ?URL/s]

https://www.fast-growing-trees.com/products/colorful-mixed-garden


GettingInfo:   1%|▋                                                                   | 1/100 [00:08<13:30,  8.19s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/colorful-mixed-garden', 'ProductName': 'Colorful Mixed Garden', 'SubHeading': "Hyacinth 'Delft Blue'; Narcissus 'Jetfire'; Tulipa 'Freeman'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0'}]
https://www.fast-growing-trees.com/products/pink-princess-philodendron


GettingInfo:   2%|█▎                                                                  | 2/100 [00:15<12:19,  7.54s/URL]

...

[{'ProductURL': 'https://www.fast-growing-trees.com/products/autumn-coral-encore-azalea', 'ProductName': 'Autumn Coral®  Encore® Azalea', 'SubHeading': "Azalea x 'Conled'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '1 reviews', 'Rating': '5.0'}]
https://www.fast-growing-trees.com/products/autumn-chiffon-encore-azalea


GettingInfo:  99%|██████████████████████████████████████████████████████████████████▎| 99/100 [13:58<00:08,  8.17s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/autumn-chiffon-encore-azalea', 'ProductName': 'Autumn Chiffon®  Encore® Azalea', 'SubHeading': "Azalea x 'Robled' PP#15862", 'Availability': 'InStock', 'Category': '3 Gallon', 'Pack': None, 'RegularPrice': '$84.95', 'SalePrice': '$69.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/howardi-ligustrum-shrub


GettingInfo: 100%|██████████████████████████████████████████████████████████████████| 100/100 [14:05<00:00,  8.46s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/howardi-ligustrum-shrub', 'ProductName': 'Howardi Ligustrum Shrub', 'SubHeading': "Ligustrum japonicum 'Howardii'", 'Availability': 'SoldOut', 'Category': None, 'Pack': None, 'RegularPrice': None, 'SalePrice': None, 'ReviewCount': '0 reviews', 'Rating': '0.0'}]


In [33]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1500:], desc="GettingInfo", unit="URL"):
    print(i)
    df = get_product_info(i)
    Product_details16.extend(df)
    print(df)
daf = pd.DataFrame(Product_details16)
daf.to_csv('FastGrowingTrees16.csv',index=False)
driver.quit()

GettingInfo:   0%|                                                                             | 0/74 [00:00<?, ?URL/s]

https://www.fast-growing-trees.com/products/hamlin-sweet-orange-tree


GettingInfo:   1%|▉                                                                    | 1/74 [00:12<15:39, 12.87s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/hamlin-sweet-orange-tree', 'ProductName': 'Hamlin Sweet Orange Tree', 'SubHeading': "Citrus sinensis 'Hamlin'", 'Availability': 'InStock', 'Category': '3-4 feet', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$109.95', 'ReviewCount': '1 reviews', 'Rating': '5.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/hamlin-sweet-orange-tree', 'ProductName': 'Hamlin Sweet Orange Tree', 'SubHeading': "Citrus sinensis 'Hamlin'", 'Availability': 'InStock', 'Category': '4-5 feet', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$119.95', 'ReviewCount': '1 reviews', 'Rating': '5.0'}]
https://www.fast-growing-trees.com/products/shiranui-mandarin-tree


GettingInfo:   3%|█▊                                                                   | 2/74 [00:26<15:53, 13.24s/URL]

...

[{'ProductURL': 'https://www.fast-growing-trees.com/products/new-gen-independence-boxwood', 'ProductName': 'NewGen Independence® Boxwood', 'SubHeading': 'Buxus NewGen Independence', 'Availability': 'InStock', 'Category': '1 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$69.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/new-gen-independence-boxwood', 'ProductName': 'NewGen Independence® Boxwood', 'SubHeading': 'Buxus NewGen Independence', 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$89.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/newgen-freedom-boxwood


GettingInfo:  99%|███████████████████████████████████████████████████████████████████ | 73/74 [11:09<00:09,  9.78s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/newgen-freedom-boxwood', 'ProductName': 'NewGen Freedom® Boxwood', 'SubHeading': 'Buxus NewGen Freedom®', 'Availability': 'InStock', 'Category': '1 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$69.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/newgen-freedom-boxwood', 'ProductName': 'NewGen Freedom® Boxwood', 'SubHeading': 'Buxus NewGen Freedom®', 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$89.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}]
https://www.fast-growing-trees.com/products/pugster-blue-butterfly-bush


GettingInfo: 100%|████████████████████████████████████████████████████████████████████| 74/74 [11:24<00:00,  9.25s/URL]

[{'ProductURL': 'https://www.fast-growing-trees.com/products/pugster-blue-butterfly-bush', 'ProductName': 'Pugster Blue® Butterfly Bush', 'SubHeading': "Buddleia davidii 'Pugster Blue'", 'Availability': 'InStock', 'Category': '1 quart', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$39.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/pugster-blue-butterfly-bush', 'ProductName': 'Pugster Blue® Butterfly Bush', 'SubHeading': "Buddleia davidii 'Pugster Blue'", 'Availability': 'InStock', 'Category': '2 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$69.95', 'ReviewCount': '0 reviews', 'Rating': '0.0'}, {'ProductURL': 'https://www.fast-growing-trees.com/products/pugster-blue-butterfly-bush', 'ProductName': 'Pugster Blue® Butterfly Bush', 'SubHeading': "Buddleia davidii 'Pugster Blue'", 'Availability': 'InStock', 'Category': '3 gallon', 'Pack': None, 'RegularPrice': None, 'SalePrice': '$99.95', 'ReviewCount': '0 re

In [32]:
len(Pro_URL)

1574

In [38]:
import os
import pandas as pd

def combine_csv_files(input_folder, output_file):
    # Get a list of all CSV files in the input folder starting with "FastGrowingTrees"
    csv_files = [file for file in os.listdir(input_folder) if file.startswith('FastGrowingTrees') and file.endswith('.csv')]

    # Initialize an empty list to store DataFrames
    data_frames = []

    # Loop through each matching CSV file and append its data to the data_frames list
    for file in csv_files:
        file_path = os.path.join(input_folder, file)
        current_data = pd.read_csv(file_path)
        print(file ,len(current_data))
        data_frames.append(current_data)

    # Concatenate all DataFrames in the list along the rows
    combined_data = pd.concat(data_frames, ignore_index=True)

    # Save the combined data to a new CSV file
    combined_data.to_csv(output_file, index=False)
    print(f"Combined data saved to {output_file}")

# Example usage
input_folder = r'C:\Users\Rahul Manocha\Documents\KDMS\WebScrapping\1st time\FastGrowingTrees'
output_file = 'FastGrowingTrees.csv'
combine_csv_files(input_folder, output_file)

FastGrowingTrees1.csv 424
FastGrowingTrees10.csv 242
FastGrowingTrees11.csv 151
FastGrowingTrees12.csv 236
FastGrowingTrees13.csv 123
FastGrowingTrees14.csv 172
FastGrowingTrees15.csv 127
FastGrowingTrees16.csv 94
FastGrowingTrees2.csv 340
FastGrowingTrees3.csv 238
FastGrowingTrees4.csv 267
FastGrowingTrees5.csv 231
FastGrowingTrees6.csv 244
FastGrowingTrees7.csv 156
FastGrowingTrees8.csv 236
FastGrowingTrees9.csv 185
Combined data saved to FastGrowingTrees.csv
